# Basic Prompt Engineering

## Step 1. Prepare Large Language Model (LLM) and Embedding Model 
---

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates') 

In [ ]:
import time
import sagemaker, boto3, json
import glob
import os
import pandas as pd
import requests
import json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

In [ ]:
!pip install --upgrade pip --quiet --disable-pip-version-check --root-user-action=ignore

In [ ]:
!pip install -qU nltk faiss-cpu langchain wikipedia pdfplumber unstructured

In [ ]:
RESTAPI_ID = "dkk7v2wvb1" # YOUR RESTAPI ID
URL = f'https://{RESTAPI_ID}.execute-api.{aws_region}.amazonaws.com/api/'.replace('"','')
LLM_URL = f"{URL}/llm/llama2_7b"
EMB_URL = f"{URL}/emb/gptj_6b"
HEADERS = {    
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

<br>

## Step 2. Ask a question to LLM without RAG
---

### Simple prompt engineering

In [ ]:
from lib_en import Llama2ContentHandlerAmazonAPIGateway
from langchain.llms import AmazonAPIGateway

params = {
    'max_new_tokens': 128,
    'top_p': 0.95,
    'temperature': 0.2,
    'return_full_text': False
}

llm = AmazonAPIGateway(api_url=LLM_URL, headers=HEADERS)
llm.content_handler = Llama2ContentHandlerAmazonAPIGateway()
llm.model_kwargs = params

In [ ]:
%%time

payload = {
    "inputs": "Generative AI is",
    "parameters": params
}
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generation"])

In [ ]:
%%time

payload = {
    "inputs": """A brief email message of Amazon SageMaker's main features

Hi everyone,

We are announcing""",
    "parameters": {"max_new_tokens": 64, "top_p": 0.9, "temperature": 0.6, "return_full_text": False}
}
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generation"])

### More complex prompts: Play the role of AWS SA


In [ ]:
architect_prompt = """
Play the role of a solution architect experienced with AWS. You are analysing customer requirements to create
well-architected solution architectures that you present to the customer. You are detailled, kind and
focussed. Given the following context

Context:
#System Requirements:
{requirements}
#Scale:
{scale}
#Features:
{features}

Describe an architecture on AWS in technical detail with sentences.
"""
prompt = architect_prompt.format(
    requirements="A website for computer advertising", 
    scale="Must handle 10k requests per second in peak. Must be globally available. Must be reponsive and fast", 
    features="Landing page describing our product. About page describing the company."
)
print(prompt)

In [ ]:
params = {
    'max_new_tokens': 100,
    'top_p': 0.95,
    'temperature': 0.8,
    'return_full_text': False
}

payload = {
    'inputs': prompt,
    'parameters': params
}

response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generation"])

### Applying LangChain

In [ ]:
llm.model_kwargs = params
print(llm(prompt))

In [ ]:
from langchain.prompts import PromptTemplate

# First we can define an exposed parameter interface to the format string
prompt = PromptTemplate(
    input_variables=["requirements", "scale", "features"],
    template=architect_prompt,
)

final_prompt = architect_prompt.format(
    requirements="External facing web application written in Javascript, global deployment",
    scale="Average of 500 requests per minute, scale events up to 3000 requests per second",
    features="Mobile website, desktop version, javascript"
)
print(final_prompt)

In [ ]:
print(llm(final_prompt))

In [ ]:
topic_recommender_prompt = "List {number} topics to write on blog posts about {topic}"

recommend_topic_prompt = PromptTemplate(
    template=topic_recommender_prompt,
    input_variables=['topic', 'number']
)

results = llm(recommend_topic_prompt.format(topic="Machine Learning", number=5))
print(results)

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
parsed_recommender_prompt = topic_recommender_prompt + "\n{format_instructions}"

parser = CommaSeparatedListOutputParser()

parsed_recommender_template = PromptTemplate(
    template=parsed_recommender_prompt,
    input_variables=['topic', 'number'],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
gen_prompt = parsed_recommender_template.format(topic='Generative AI', number=5)
print(gen_prompt)

In [ ]:
output = llm(gen_prompt)
print(output)